Correct way to open:
1. ubuntu promt
2. cd /home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/python\ code
3. code .

Mapbox api

pk.eyJ1Ijoiam9obm55Ym95NjY5NyIsImEiOiJjbTgyMzZkbjUxZHF2MmlzYTByc3pxZmw0In0.7RASB6M_AczC7q8dvFPWBQ

In [109]:
#Taipei_urban_masterplan.geojson:
#{
#"type": "FeatureCollection",
#"name": "Taipei_urban_masterplan",
#"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
#"features": [
#{ "type": "Feature", "properties": { "編號": "1", "圖層": "32", "顏色": "19", "街廓編號": null, "分區代碼": "PEA", "分區簡稱": "公", "使用分區": "公園用地", "分區說明": null, "原屬分區": null, "變更前代碼": null, "變更前簡稱": null, "變更前分區": null, "Category": "City_Open_Area", "Area": 823.19 }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 121.528633581995919, 25.047796387544285 ], [ 121.528533470499838, 25.048123199000166 ], [ 121.528553569927254, 25.048141923941568 ], [ 121.528906514957143, 25.048059586583932 ], [ 121.528919131953018, 25.048015927027393 ], [ 121.528876127568978, 25.047978288611517 ], [ 121.528844065787951, 25.047950226128741 ], [ 121.528706848633234, 25.047828447317585 ], [ 121.528662044074167, 25.047789289499065 ], [ 121.528633581995919, 25.047796387544285 ] ] ] ] } },
#the data I want to present in kepler are "Category" & "Area"

Test1

In [110]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString
from shapely.ops import unary_union, linemerge
import cugraph
import cudf  # Add this import
import os
from tqdm import tqdm
import pickle
import hashlib
from keplergl import KeplerGl
import json
import webbrowser

In [111]:
# Constants
CHECKPOINT_DIR = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/checkpoints"
SUBGRAPH_DIR = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/Neighborhood_subgraph"
CONFIG_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/kepler.gl.json"
TEMP_HTML_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/temp_walkability_map.html"
URBAN_MASTERPLAN_PATH = "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/Taipei_urban_masterplan.geojson"
MAPBOX_ACCESS_TOKEN = "pk.eyJ1Ijoiam9obm55Ym95NjY5NyIsImEiOiJjbTgyMzZkbjUxZHF2MmlzYTByc3pxZmw0In0.7RASB6M_AczC7q8dvFPWBQ"
WORKING_CRS = 'EPSG:3826'  # TWD97 / TM2 zone 121 (meters)

# Ensure directories exist
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(SUBGRAPH_DIR, exist_ok=True)

def validate_geometry(gdf, name):
    """Validate and repair geometries in a GeoDataFrame."""
    print(f"Validating geometries for {name}...")
    gdf['geometry'] = gdf.geometry.make_valid()
    invalid = gdf[~gdf.geometry.is_valid]
    if not invalid.empty:
        print(f"Found {len(invalid)} invalid geometries in {name}. Attempting to repair...")
        gdf = gdf[gdf.geometry.is_valid]
    gdf = gdf[~gdf.geometry.is_empty]
    print(f"{name} after validation: {len(gdf)} rows")
    return gdf

def load_and_prepare_data():
    """Load and preprocess all geospatial datasets."""
    print("Checkpoint 1: Loading and preparing data...")

    # Define file paths
    paths = {
        'neighborhoods': "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/neighborhoods_with_ndvi_numerical.geojson",
        'buildings': "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/Taipei_Buildings_fulldata.geojson",
        'roads': "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/taipei_segments_cleaned_verified.geoparquet",
        'trees': "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/taipei_land.geoparquet",
        'transit': "/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data/taipei_infrastructure.geoparquet",
        'urban_masterplan': URBAN_MASTERPLAN_PATH
    }

    # Load datasets
    data = {}
    for name, path in tqdm(paths.items(), desc="Loading files"):
        if not os.path.exists(path):
            raise FileNotFoundError(f"File not found: {path}")
        if path.endswith('.geojson'):
            gdf = gpd.read_file(path, encoding='utf-8-sig')
        else:
            gdf = gpd.read_parquet(path)
        gdf = gdf.to_crs(WORKING_CRS)
        gdf = validate_geometry(gdf, name)
        data[name] = gdf

    # Clean and preprocess specific datasets
    # Trees: Filter for trees and convert polylines to points
    trees_gdf = data['trees']
    trees_gdf = trees_gdf[trees_gdf['subtype'] == 'tree']
    tree_points = trees_gdf[trees_gdf.geometry.geom_type == 'Point']
    tree_polylines = trees_gdf[trees_gdf.geometry.geom_type.isin(['LineString', 'MultiLineString'])]
    
    converted_points = []
    for geom in tree_polylines.geometry:
        if geom.geom_type == 'MultiLineString':
            geom = linemerge(geom)
        if geom.geom_type != 'LineString' or geom.length <= 0:
            continue
        distance = 0
        while distance <= geom.length:
            point = geom.interpolate(distance)
            converted_points.append({'geometry': point})
            distance += 7  # Spacing of 7 meters
    if converted_points:
        converted_points_gdf = gpd.GeoDataFrame(converted_points, crs=WORKING_CRS)
        all_trees = pd.concat([tree_points.geometry, converted_points_gdf.geometry], ignore_index=True)
    else:
        all_trees = tree_points.geometry
    data['trees'] = gpd.GeoDataFrame(geometry=all_trees, crs=WORKING_CRS)
    print(f"Total number of trees after processing: {len(data['trees'])}")

    # Transit: Filter for bus stops and points only
    transit_gdf = data['transit']
    transit_gdf = transit_gdf[transit_gdf['class'].isin(['stop_position', 'bus_stop'])]
    transit_gdf = transit_gdf[transit_gdf.geometry.geom_type == 'Point']
    transit_gdf = transit_gdf[~transit_gdf.geometry.isna()]
    data['transit'] = transit_gdf
    print(f"Total number of transit points after filtering: {len(data['transit'])}")

    # Buildings: Clean building types
    buildings_gdf = data['buildings']
    buildings_gdf['area_m2'] = buildings_gdf.geometry.area.round(1)
    buildings_gdf['building'] = buildings_gdf['building'].fillna('unknown').replace('yes', 'unknown')
    data['buildings'] = buildings_gdf

    # Roads: Add length
    roads_gdf = data['roads']
    roads_gdf['length_m'] = roads_gdf.geometry.length.round(1)
    data['roads'] = roads_gdf

    return data

def compute_data_hash(gdf):
    """Compute a hash of a GeoDataFrame for caching."""
    df_hashable = gdf.drop(columns=['geometry']).astype(str).fillna('missing')
    return hashlib.md5(pd.util.hash_pandas_object(df_hashable).values.tobytes()).hexdigest()

def calculate_neighborhood_features(neighborhoods_gdf, trees_gdf, transit_gdf, urban_masterplan_gdf):
    """Calculate features for each neighborhood (tree count, transit count, land use)."""
    print("Calculating neighborhood features...")

    # Initialize feature columns
    neighborhoods_gdf['tree_count'] = 0
    neighborhoods_gdf['transit_count'] = 0
    unique_categories = urban_masterplan_gdf['Category'].unique()
    for category in unique_categories:
        attr_name_m2 = f"land_use_{category.lower().replace('_', '_')}_m2"
        attr_name_percent = f"land_use_{category.lower().replace('_', '_')}_percent"
        neighborhoods_gdf[attr_name_m2] = 0.0
        neighborhoods_gdf[attr_name_percent] = 0.0

    # Cache setup
    cache_dir = CHECKPOINT_DIR
    urban_seg_cache_path = os.path.join(cache_dir, "urban_seg_selected_neighborhoods.pkl")
    urban_seg_dict_path = os.path.join(cache_dir, "urban_seg_dict.pkl")
    urban_seg_hash_path = os.path.join(cache_dir, "urban_seg_hash.txt")

    input_data_hash = compute_data_hash(urban_masterplan_gdf) + compute_data_hash(neighborhoods_gdf)

    cached_results_exist = False
    if os.path.exists(urban_seg_cache_path) and os.path.exists(urban_seg_dict_path) and os.path.exists(urban_seg_hash_path):
        with open(urban_seg_hash_path, 'r') as f:
            cached_hash = f.read()
        if cached_hash == input_data_hash:
            with open(urban_seg_cache_path, 'rb') as f:
                neighborhoods_gdf = pickle.load(f)
            with open(urban_seg_dict_path, 'rb') as f:
                urban_plan_to_neighborhoods = pickle.load(f)
            print("Loaded cached urban masterplan segregation results.")
            cached_results_exist = True

    if not cached_results_exist:
        # Calculate tree and transit counts using spatial join
        print("Counting trees per neighborhood...")
        trees_joined = gpd.sjoin(trees_gdf, neighborhoods_gdf, how='left', predicate='within')
        tree_counts = trees_joined.groupby('index_right').size()
        neighborhoods_gdf['tree_count'] = neighborhoods_gdf.index.map(tree_counts).fillna(0).astype(int)

        print("Counting transit points per neighborhood...")
        transit_joined = gpd.sjoin(transit_gdf, neighborhoods_gdf, how='left', predicate='within')
        transit_counts = transit_joined.groupby('index_right').size()
        neighborhoods_gdf['transit_count'] = neighborhoods_gdf.index.map(transit_counts).fillna(0).astype(int)

        # Calculate land use areas
        urban_plan_to_neighborhoods = {}
        for idx in tqdm(range(len(urban_masterplan_gdf)), desc="Segregating urban masterplan"):
            urban_plan_to_neighborhoods[idx] = []
            masterplan_row = urban_masterplan_gdf.iloc[idx]
            masterplan_geom = masterplan_row.geometry
            masterplan_gdf = gpd.GeoDataFrame([masterplan_row], geometry=[masterplan_geom], crs=WORKING_CRS)

            intersections = gpd.overlay(neighborhoods_gdf, masterplan_gdf, how='intersection', keep_geom_type=False)
            if intersections.empty:
                continue

            # Compute the masterplan area directly from the geometry (in square meters)
            masterplan_area = masterplan_geom.area
            if masterplan_area <= 0:
                continue

            for _, intersection_row in intersections.iterrows():
                neigh_idx = intersection_row.name
                intersection_geom = intersection_row.geometry
                if intersection_geom.is_empty:
                    continue

                intersection_area = intersection_geom.area
                if intersection_area > 0:
                    category = masterplan_row['Category']
                    # Prorate based on geometry areas (both in square meters)
                    prorated_area = (intersection_area / masterplan_area) * masterplan_area
                    attr_name_m2 = f"land_use_{category.lower().replace('_', '_')}_m2"
                    neighborhoods_gdf.at[neigh_idx, attr_name_m2] += prorated_area
                    urban_plan_to_neighborhoods[idx].append((neighborhoods_gdf['LIE_NAME'].iloc[neigh_idx], intersection_area))

        # Calculate total allocated area per neighborhood to normalize percentages
        total_allocated_area = pd.Series(0.0, index=neighborhoods_gdf.index)
        for category in unique_categories:
            attr_name_m2 = f"land_use_{category.lower().replace('_', '_')}_m2"
            total_allocated_area += neighborhoods_gdf[attr_name_m2]

        # Calculate percentages, normalizing to ensure they sum to 100%
        neighborhood_areas = neighborhoods_gdf.geometry.area
        for category in unique_categories:
            attr_name_m2 = f"land_use_{category.lower().replace('_', '_')}_m2"
            attr_name_percent = f"land_use_{category.lower().replace('_', '_')}_percent"
            # First calculate raw percentage based on neighborhood area
            raw_percent = (neighborhoods_gdf[attr_name_m2] / neighborhood_areas * 100).round(4)
            # Normalize by the total allocated area
            normalized_percent = (neighborhoods_gdf[attr_name_m2] / total_allocated_area * 100).round(4)
            # Use raw percentage if total allocated area is less than neighborhood area, otherwise normalize
            neighborhoods_gdf[attr_name_percent] = np.where(
                total_allocated_area <= neighborhood_areas,
                raw_percent,
                normalized_percent
            )
            neighborhoods_gdf[attr_name_percent] = neighborhoods_gdf[attr_name_percent].where(neighborhood_areas > 0, 0)

        # Cache results
        with open(urban_seg_cache_path, 'wb') as f:
            pickle.dump(neighborhoods_gdf, f)
        with open(urban_seg_dict_path, 'wb') as f:
            pickle.dump(urban_plan_to_neighborhoods, f)
        with open(urban_seg_hash_path, 'w') as f:
            f.write(input_data_hash)
        print("Cached urban masterplan segregation results.")

    # Print results
    print("Tree, transit, and urban masterplan counts per neighborhood:")
    for idx, row in neighborhoods_gdf.iterrows():
        print(f"- {row['LIE_NAME']}: {row['tree_count']} trees, {row['transit_count']} transit points")
        for category in unique_categories:
            attr_name_m2 = f"land_use_{category.lower().replace('_', '_')}_m2"
            attr_name_percent = f"land_use_{category.lower().replace('_', '_')}_percent"
            print(f"  - {category} Area: {row[attr_name_m2]:.2f}m² ({row[attr_name_percent]:.4f}%)")

    return neighborhoods_gdf, urban_plan_to_neighborhoods

In [112]:
def build_graph(data, urban_plan_to_neighborhoods):
    print("Checkpoint 2: Building graph network...")
    neighborhoods_gdf = data['neighborhoods']
    buildings_gdf = data['buildings']
    roads_gdf = data['roads']
    trees_gdf = data['trees']
    transit_gdf = data['transit']
    urban_masterplan_gdf = data['urban_masterplan']

    subgraphs = {}
    road_network = cugraph.Graph(directed=False)
    road_network_nodes = []

    # Define road class weights outside the loop
    road_class_weights = {
        'footway': 0.5, 'pedestrian': 0.5, 'cycleway': 0.6, 'steps': 0.6, 'living_street': 0.6,
        'path': 0.7, 'track': 0.7, 'residential': 0.8, 'service': 1.0, 'unclassified': 1.2,
        'tertiary': 1.5, 'secondary': 2.0, 'primary': 2.5, 'highway': 3.5, 'motorway': 2.5,
        'trunk': 2.5, 'unknown': 1.0
    }

    # Checkpoint file to track progress
    checkpoint_file = os.path.join(CHECKPOINT_DIR, "build_graph_checkpoint.txt")
    completed_indices = set()
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            completed_indices = set(int(line.strip()) for line in f if line.strip().isdigit())
        print(f"Loaded {len(completed_indices)} completed neighborhoods from checkpoint.")

    # Check for existing .pkl files and determine which need processing
    to_process = []
    for idx in range(len(neighborhoods_gdf)):
        lie_name = neighborhoods_gdf['LIE_NAME'].iloc[idx]
        subgraph_path = os.path.join(SUBGRAPH_DIR, f"subgraph_{lie_name}.pkl")
        if idx in completed_indices and os.path.exists(subgraph_path):
            with open(subgraph_path, 'rb') as f:
                subgraph_data = pickle.load(f)
                subgraphs[lie_name] = subgraph_data
                # Extract road nodes from loaded subgraph
                nodes_df = subgraph_data['nodes'].to_pandas()
                road_nodes_df = nodes_df[nodes_df['type'] == 'road']
                for _, row in road_nodes_df.iterrows():
                    road_network_nodes.append({
                        'vertex': row['vertex'],
                        'type': 'road',
                        'road_class': row['road_class'],
                        'length_m': row['length_m']
                    })
            print(f"Loaded existing subgraph for {lie_name} from {subgraph_path}")
        else:
            to_process.append(idx)

    # Process missing or incomplete neighborhoods
    for idx in tqdm(to_process, desc="Processing neighborhoods", total=len(to_process)):
        lie_name = neighborhoods_gdf['LIE_NAME'].iloc[idx]
        G_sub = cugraph.Graph(directed=False)
        all_nodes = []
        all_edges = []

        # Neighborhood node
        node_id = f"neighborhood_{lie_name}"
        node_data = {
            'vertex': node_id,
            'type': 'neighborhood',
            'lie_name': lie_name,
            'sect_name': neighborhoods_gdf['SECT_NAME'].iloc[idx],
            'population': neighborhoods_gdf['2024population'].iloc[idx],
            'land_use_residential_percent': neighborhoods_gdf['land_use_residential_percent'].iloc[idx],
            'land_use_commercial_percent': neighborhoods_gdf['land_use_commercial_percent'].iloc[idx],
            'land_use_education_percent': neighborhoods_gdf['land_use_education_percent'].iloc[idx],
            'ndvi_mean': neighborhoods_gdf['ndvi_mean'].iloc[idx],
            'ndvi_median': neighborhoods_gdf['ndvi_median'].iloc[idx],
            'tree_count': neighborhoods_gdf['tree_count'].iloc[idx],
            'transit_count': neighborhoods_gdf['transit_count'].iloc[idx]
        }
        unique_categories = urban_masterplan_gdf['Category'].unique()
        for category in unique_categories:
            attr_name_m2 = f"land_use_{category.lower().replace('_', '_')}_m2"
            attr_name_percent = f"land_use_{category.lower().replace('_', '_')}_percent"
            node_data[attr_name_m2] = neighborhoods_gdf[attr_name_m2].iloc[idx]
            node_data[attr_name_percent] = neighborhoods_gdf[attr_name_percent].iloc[idx]
        all_nodes.append(node_data)

        # Buffer and spatial filtering
        buffer_distance = 200
        neigh_geom = neighborhoods_gdf.geometry.iloc[idx]
        neigh_buffer = neigh_geom.buffer(buffer_distance)

        relevant_buildings = buildings_gdf[buildings_gdf.geometry.within(neigh_buffer)]
        relevant_roads = roads_gdf[roads_gdf.geometry.intersects(neigh_buffer)]
        relevant_trees = trees_gdf[trees_gdf.geometry.within(neigh_buffer)]
        relevant_transit = transit_gdf[transit_gdf.geometry.within(neigh_buffer)]

        # Building nodes
        building_nodes = {}
        for b_idx, building in relevant_buildings.iterrows():
            node_id = f"building_{b_idx}"
            building_data = {
                'vertex': node_id,
                'type': 'building',
                'building_type': building['building'],
                'area_m2': building['area_m2'],
                'age': building.get('屋齡', '<NA>'),
                'height': building.get('建物高度', '<NA>'),
                'floors': building.get('地上層數', '<NA>'),
                'structure_type': building.get('構造種類', 'Unknown'),
                'usage_zone': building.get('使用分區', 'Unknown')
            }
            all_nodes.append(building_data)
            building_nodes[node_id] = building.geometry

        # Road nodes
        road_nodes = {}
        road_points = []
        for r_idx, road in relevant_roads.iterrows():
            geom = road.geometry
            if geom.geom_type == 'LineString':
                start_point = Point(geom.coords[0])
                end_point = Point(geom.coords[-1])
                road_points.extend([(f"road_start_{r_idx}", start_point), (f"road_end_{r_idx}", end_point)])
            elif geom.geom_type == 'MultiLineString':
                for i, line in enumerate(geom.geoms):
                    start_point = Point(line.coords[0])
                    end_point = Point(line.coords[-1])
                    road_points.extend([(f"road_start_{r_idx}_{i}", start_point), (f"road_end_{r_idx}_{i}", end_point)])

        for node_id, geom in road_points:
            r_idx = int(node_id.split('_')[2])
            road_data = {
                'vertex': node_id,
                'type': 'road',
                'road_class': roads_gdf['class'].iloc[r_idx] if pd.notna(roads_gdf['class'].iloc[r_idx]) else 'unknown',
                'length_m': roads_gdf['length_m'].iloc[r_idx]
            }
            all_nodes.append(road_data)
            road_nodes[node_id] = geom
            road_network_nodes.append(road_data)

        # Tree nodes
        tree_nodes = {}
        for t_idx, tree in relevant_trees.iterrows():
            node_id = f"tree_{t_idx}"
            tree_data = {'vertex': node_id, 'type': 'tree'}
            all_nodes.append(tree_data)
            tree_nodes[node_id] = tree.geometry

        # Transit nodes
        transit_nodes = {}
        for t_idx, transit in relevant_transit.iterrows():
            node_id = f"transit_{t_idx}"
            transit_data = {
                'vertex': node_id,
                'type': 'transit',
                'class': transit['class']
            }
            all_nodes.append(transit_data)
            transit_nodes[node_id] = transit.geometry

        # Urban plan nodes and edges
        urban_plan_nodes = {}
        for urban_idx in urban_plan_to_neighborhoods:
            if idx in [neigh_idx for _, neigh_idx in urban_plan_to_neighborhoods[urban_idx]]:
                node_id = f"urban_plan_{urban_idx}"
                urban_plan_data = {
                    'vertex': node_id,
                    'type': 'urban_plan',
                    'category': urban_masterplan_gdf['Category'].iloc[urban_idx],
                    'area': urban_masterplan_gdf.geometry.iloc[urban_idx].area
                }
                all_nodes.append(urban_plan_data)
                urban_plan_nodes[node_id] = urban_masterplan_gdf.geometry.iloc[urban_idx]
                distance = neigh_geom.centroid.distance(urban_masterplan_gdf.geometry.iloc[urban_idx].centroid)
                all_edges.append({'src': node_id, 'dst': f"neighborhood_{lie_name}", 'weight': distance, 'type': 'urban_plan'})
                all_edges.append({'src': f"neighborhood_{lie_name}", 'dst': node_id, 'weight': distance, 'type': 'urban_plan'})

                urban_plan_buffer = urban_masterplan_gdf.geometry.iloc[urban_idx].buffer(50)
                for building_node, build_geom in building_nodes.items():
                    if build_geom.within(urban_plan_buffer):
                        distance = urban_masterplan_gdf.geometry.iloc[urban_idx].centroid.distance(build_geom)
                        all_edges.append({'src': node_id, 'dst': building_node, 'weight': distance, 'type': 'urban_plan_to_building'})
                        all_edges.append({'src': building_node, 'dst': node_id, 'weight': distance, 'type': 'urban_plan_to_building'})
                for road_node, road_geom in road_nodes.items():
                    if road_geom.within(urban_plan_buffer):
                        distance = urban_masterplan_gdf.geometry.iloc[urban_idx].centroid.distance(road_geom)
                        all_edges.append({'src': node_id, 'dst': road_node, 'weight': distance, 'type': 'urban_plan_to_road'})
                        all_edges.append({'src': road_node, 'dst': node_id, 'weight': distance, 'type': 'urban_plan_to_road'})
                for tree_node, tree_geom in tree_nodes.items():
                    if tree_geom.within(urban_plan_buffer):
                        distance = urban_masterplan_gdf.geometry.iloc[urban_idx].centroid.distance(tree_geom)
                        all_edges.append({'src': node_id, 'dst': tree_node, 'weight': distance, 'type': 'urban_plan_to_tree'})
                        all_edges.append({'src': tree_node, 'dst': node_id, 'weight': distance, 'type': 'urban_plan_to_tree'})
                for transit_node, transit_geom in transit_nodes.items():
                    if transit_geom.within(urban_plan_buffer):
                        distance = urban_masterplan_gdf.geometry.iloc[urban_idx].centroid.distance(transit_geom)
                        all_edges.append({'src': node_id, 'dst': transit_node, 'weight': distance, 'type': 'urban_plan_to_transit'})
                        all_edges.append({'src': transit_node, 'dst': node_id, 'weight': distance, 'type': 'urban_plan_to_transit'})

        # Neighborhood edges
        neigh_node = f"neighborhood_{lie_name}"
        for building_node, build_geom in building_nodes.items():
            distance = neigh_geom.distance(build_geom)
            all_edges.append({'src': neigh_node, 'dst': building_node, 'weight': distance, 'type': 'walk'})
            all_edges.append({'src': building_node, 'dst': neigh_node, 'weight': distance, 'type': 'walk'})

        for road_node, road_geom in road_nodes.items():
            distance = neigh_geom.distance(road_geom)
            all_edges.append({'src': neigh_node, 'dst': road_node, 'weight': distance, 'type': 'walk'})
            all_edges.append({'src': road_node, 'dst': neigh_node, 'weight': distance, 'type': 'walk'})

        for building_node, build_geom in building_nodes.items():
            build_buffer = build_geom.buffer(50)
            for road_node, road_geom in road_nodes.items():
                if road_geom.within(build_buffer):
                    distance = build_geom.distance(road_geom)
                    all_edges.append({'src': building_node, 'dst': road_node, 'weight': distance, 'type': 'walk'})
                    all_edges.append({'src': road_node, 'dst': building_node, 'weight': distance, 'type': 'walk'})

        for tree_node, tree_geom in tree_nodes.items():
            distance = neigh_geom.distance(tree_geom)
            all_edges.append({'src': neigh_node, 'dst': tree_node, 'weight': distance, 'type': 'natural'})
            all_edges.append({'src': tree_node, 'dst': neigh_node, 'weight': distance, 'type': 'natural'})

        for transit_node, transit_geom in transit_nodes.items():
            distance = neigh_geom.distance(transit_geom)
            all_edges.append({'src': neigh_node, 'dst': transit_node, 'weight': distance, 'type': 'transit'})
            all_edges.append({'src': transit_node, 'dst': neigh_node, 'weight': distance, 'type': 'transit'})

        # Road network edges within the loop
        tolerance = 10
        road_node_list = list(road_nodes.items())
        for i, (node1_id, geom1) in enumerate(road_node_list):
            for j, (node2_id, geom2) in enumerate(road_node_list[i+1:], start=i+1):
                if geom1.distance(geom2) <= tolerance:
                    road1_idx = int(node1_id.split('_')[2])
                    road2_idx = int(node2_id.split('_')[2])
                    class1 = roads_gdf['class'].iloc[road1_idx] if pd.notna(roads_gdf['class'].iloc[road1_idx]) else 'unknown'
                    class2 = roads_gdf['class'].iloc[road2_idx] if pd.notna(roads_gdf['class'].iloc[road2_idx]) else 'unknown'
                    weight1 = road_class_weights.get(class1, road_class_weights['unknown'])
                    weight2 = road_class_weights.get(class2, road_class_weights['unknown'])
                    distance = geom1.distance(geom2)
                    weight = distance * (weight1 + weight2) / 2
                    all_edges.append({'src': node1_id, 'dst': node2_id, 'weight': weight, 'type': 'road'})
                    all_edges.append({'src': node2_id, 'dst': node1_id, 'weight': weight, 'type': 'road'})

        # Convert to DataFrames
        nodes_df = cudf.from_pandas(pd.DataFrame(all_nodes))
        edges_df = cudf.from_pandas(pd.DataFrame(all_edges))

        print(f"Edges DataFrame for {lie_name} before initializing graph:")
        print(f"Shape: {edges_df.shape}")
        print(f"Columns: {edges_df.columns.tolist()}")
        print(f"Any NaN in edges_df:\n{edges_df.isna().any()}")
        edges_df = edges_df.dropna()
        edges_df = edges_df.drop_duplicates()
        print(f"Shape after cleaning: {edges_df.shape}")

        # Graph construction
        if not edges_df.empty:
            G_sub.from_cudf_edgelist(edges_df, source='src', destination='dst', edge_attr='weight', symmetrize=False)
            nodes_from_graph = G_sub.nodes()
            if isinstance(nodes_from_graph, cudf.Series):
                nodes_from_graph = cudf.DataFrame({'vertex': nodes_from_graph})
            nodes_from_graph_pandas = nodes_from_graph.to_pandas()
            G_sub_nodes = nodes_from_graph_pandas.merge(pd.DataFrame(all_nodes), on='vertex', how='left')
            G_sub._nodes = cudf.from_pandas(G_sub_nodes)
        else:
            G_sub._nodes = nodes_df

        # Save subgraph data using original edges_df to preserve 'weight'
        subgraph_data = {
            'nodes': G_sub._nodes,
            'edges': edges_df  # Use edges_df instead of G_sub.edgelist.edgelist_df
        }
        subgraphs[lie_name] = subgraph_data
        print(f"Subgraph for {lie_name}: {G_sub.number_of_nodes()} nodes, {G_sub.number_of_edges()} edges")

        subgraph_path = os.path.join(SUBGRAPH_DIR, f"subgraph_{lie_name}.pkl")
        with open(subgraph_path, 'wb') as f:
            pickle.dump(subgraph_data, f)
        
        # Update checkpoint
        with open(checkpoint_file, 'a') as f:
            f.write(f"{idx}\n")
        print(f"Checkpoint saved for {lie_name} at index {idx}")

    # Road network construction with progress bar
    print("Building road network for inter-neighborhood connections...")
    tolerance = 10
    road_network_edges = []

    # Deduplicate road_network_nodes based on 'vertex'
    if road_network_nodes:
        # Convert to DataFrame and drop duplicates
        road_nodes_df = pd.DataFrame(road_network_nodes).drop_duplicates(subset='vertex')
        print(f"Number of unique road nodes after deduplication: {len(road_nodes_df)}")
        road_node_list = road_nodes_df.to_dict('records')
        total_nodes = len(road_node_list)
        # Use tqdm for the outer loop to show progress and estimated time
        for i in tqdm(range(total_nodes), desc="Building road network edges", total=total_nodes):
            node1 = road_node_list[i]
            for j in range(i + 1, total_nodes):
                node2 = road_node_list[j]
                geom1 = roads_gdf.geometry.iloc[int(node1['vertex'].split('_')[2])]
                geom2 = roads_gdf.geometry.iloc[int(node2['vertex'].split('_')[2])]
                if geom1.distance(geom2) <= tolerance:
                    class1 = node1['road_class']
                    class2 = node2['road_class']
                    weight1 = road_class_weights.get(class1, road_class_weights['unknown'])
                    weight2 = road_class_weights.get(class2, road_class_weights['unknown'])
                    distance = geom1.distance(geom2)
                    weight = distance * (weight1 + weight2) / 2
                    road_network_edges.append({'src': node1['vertex'], 'dst': node2['vertex'], 'weight': weight, 'type': 'road'})
                    road_network_edges.append({'src': node2['vertex'], 'dst': node1['vertex'], 'weight': weight, 'type': 'road'})
    else:
        road_node_list = []
        print("No road nodes to process for inter-neighborhood connections.")

    road_nodes_df = cudf.from_pandas(pd.DataFrame(road_node_list) if road_node_list else pd.DataFrame())
    if road_network_edges:
        road_edges_df = cudf.from_pandas(pd.DataFrame(road_network_edges))
        road_edges_df = road_edges_df.dropna()
        road_edges_df = road_edges_df.drop_duplicates()
        road_network.from_cudf_edgelist(road_edges_df, source='src', destination='dst', edge_attr='weight', symmetrize=False)
        road_network._nodes = road_nodes_df
    else:
        if not road_nodes_df.empty:
            road_network._nodes = road_nodes_df
        else:
            road_network._nodes = cudf.DataFrame()

    road_network_data = {
        'nodes': road_network._nodes,
        'edges': cudf.from_pandas(pd.DataFrame(road_network_edges)) if road_network_edges else None
    }
    with open(os.path.join(CHECKPOINT_DIR, "road_network.pkl"), 'wb') as f:
        pickle.dump(road_network_data, f)

    # Safely print road network stats
    num_nodes = road_network.number_of_nodes() if road_network._nodes is not None and not road_network._nodes.empty else 0
    num_edges = len(road_network_edges) if road_network_edges else 0
    print(f"Total road network nodes: {num_nodes}, edges: {num_edges}")
    return subgraphs, road_network_data

In [113]:
def calculate_walkability(subgraphs, neighborhoods_gdf):
    """Calculate walkability scores for each neighborhood."""
    print("Checkpoint 3: Calculating walkability scores...")
    walkability_scores = {}

    for lie_name, subgraph_data in tqdm(subgraphs.items(), desc="Calculating walkability"):
        G_sub = cugraph.Graph(directed=False)
        G_sub._nodes = subgraph_data['nodes']
        if subgraph_data['edges'] is not None:
            # Use the preserved edges_df with 'weight' column
            G_sub.from_cudf_edgelist(subgraph_data['edges'], source='src', destination='dst', edge_attr='weight', symmetrize=False)
        else:
            G_sub._nodes = subgraph_data['nodes']

        nodes_df = G_sub._nodes
        for idx in range(len(nodes_df)):
            node_data = nodes_df.iloc[idx].to_pandas()
            if node_data['type'] == 'neighborhood':
                residential = node_data['land_use_residential_percent']
                commercial = node_data['land_use_commercial_percent']
                education = node_data['land_use_education_percent']
                ndvi = node_data['ndvi_mean']
                tree_count = node_data['tree_count']
                transit_count = node_data['transit_count']
                open_area = node_data.get('land_use_city_open_area_m2', 0.0)

                land_use_score = (residential * 0.4 + commercial * 0.3 + education * 0.2) / 100
                ndvi_score = ndvi * 0.5 if pd.notna(ndvi) else 0.0
                tree_score = min(1.0, tree_count / 100) * 0.2
                transit_score = min(1.0, transit_count / 20) * 0.2
                open_space_score = min(1.0, open_area / 10000) * 0.2
                walkability = min(1.0, land_use_score + (ndvi_score * 0.4) + tree_score + transit_score + open_space_score)
                walkability_scores[lie_name] = walkability

                nodes_df['walkability'] = nodes_df['walkability'].astype(float) if 'walkability' in nodes_df.columns else 0.0
                nodes_df.loc[nodes_df['vertex'] == node_data['vertex'], 'walkability'] = walkability

                neighborhoods_gdf.loc[neighborhoods_gdf['LIE_NAME'] == lie_name, 'walkability'] = walkability

        subgraph_data['nodes'] = nodes_df
        subgraph_path = os.path.join(SUBGRAPH_DIR, f"subgraph_{lie_name}_with_walkability.pkl")
        with open(subgraph_path, 'wb') as f:
            pickle.dump(subgraph_data, f)

    with open(os.path.join(CHECKPOINT_DIR, "walkability_scores.pkl"), 'wb') as f:
        pickle.dump(walkability_scores, f)

    print("Walkability scores calculated.")
    return subgraphs

In [114]:
def reconstruct_full_graph(subgraphs, road_network_data):
    """Reconstruct the full graph from subgraphs and road network."""
    print("Reconstructing full graph...")
    G = cugraph.Graph(directed=False)

    # Reconstruct and add each subgraph
    for lie_name, subgraph_data in subgraphs.items():
        G_sub = cugraph.Graph(directed=False)
        G_sub._nodes = subgraph_data['nodes']
        if subgraph_data['edges'] is not None:
            G_sub.from_cudf_edgelist(subgraph_data['edges'], source='src', destination='dst', edge_attr='weight', symmetrize=False)
        G.add_nodes_from(G_sub._nodes)
        if G_sub.number_of_edges() > 0:
            G.add_edges_from(G_sub.edgelist.edgelist_df)

    # Reconstruct the road network
    road_network = cugraph.Graph(directed=False)
    road_network._nodes = road_network_data['nodes']
    if road_network_data['edges'] is not None:
        road_network.from_cudf_edgelist(road_network_data['edges'], source='src', destination='dst', edge_attr='weight', symmetrize=False)

    # Add road network to the full graph
    G.add_nodes_from(road_network._nodes)
    if road_network.number_of_edges() > 0:
        G.add_edges_from(road_network.edgelist.edgelist_df)

    print(f"Full graph reconstructed: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    return G

In [ ]:
def create_interactive_map(G, data):
    """Create an interactive Kepler.gl map."""
    print("Generating interactive Kepler.gl map...")
    neighborhoods_gdf = data['neighborhoods'].to_crs('EPSG:4326')
    buildings_gdf = data['buildings'].to_crs('EPSG:4326')
    roads_gdf = data['roads'].to_crs('EPSG:4326')
    trees_gdf = data['trees'].to_crs('EPSG:4326')
    transit_gdf = data['transit'].to_crs('EPSG:4326')
    urban_masterplan_gdf = data['urban_masterplan'].to_crs('EPSG:4326')

    # Add walkability scores to neighborhoods
    nodes_df = G._nodes  # Access the nodes directly
    for idx in range(len(nodes_df)):
        node_data = nodes_df.iloc[idx].to_pandas()
        if node_data['type'] == 'neighborhood':
            lie_name = node_data['lie_name']
            walkability = round(node_data.get('walkability', 0), 2)
            transit_count = node_data['transit_count']
            neighborhoods_gdf.loc[neighborhoods_gdf['LIE_NAME'].str.strip() == lie_name.strip(), 'walkability'] = walkability
            neighborhoods_gdf.loc[neighborhoods_gdf['LIE_NAME'].str.strip() == lie_name.strip(), 'transit_count'] = transit_count

    # Prepare data for Kepler.gl
    neighborhoods_data = neighborhoods_gdf.drop(columns=['geometry']).copy()
    for col in neighborhoods_data.columns:
        if col not in ['geometry', 'LIE_NAME', 'SECT_NAME']:
            neighborhoods_data[col] = pd.to_numeric(neighborhoods_data[col], errors='coerce').round(4)
    geojson_data = {
        'type': 'FeatureCollection',
        'features': [
            {
                'type': 'Feature',
                'properties': {k: v for k, v in row.drop('geometry').to_dict().items() if pd.notna(v) and v != 0},
                'geometry': row['geometry'].__geo_interface__ if row['geometry'] is not None else None
            }
            for _, row in neighborhoods_gdf.iterrows()
        ]
    }

    buildings_data = buildings_gdf[['full_id', 'osm_id', 'building', '屋齡', '建物高度', '地上層數', '構造種類', '使用分區', 'geometry', 'area_m2']].copy()
    buildings_data['建物高度'] = pd.to_numeric(buildings_data['建物高度'], errors='coerce').fillna(10).round(1)
    buildings_data['地上層數'] = pd.to_numeric(buildings_data['地上層數'], errors='coerce').fillna(3).round(0)

    roads_data = roads_gdf[['class', 'length_m', 'geometry']].copy()
    roads_data['class'] = roads_data['class'].fillna('unknown')
    roads_data['length_m'] = roads_data['length_m'].round(1)

    trees_data = trees_gdf[['geometry']].copy()
    trees_data['height_m'] = 10

    transit_data = transit_gdf[['id', 'class', 'geometry']].copy()
    transit_data['size'] = 10

    # Include the original 'Area' column from urban_masterplan
    urban_masterplan_data = urban_masterplan_gdf[['Category', 'Area', 'geometry']].copy()
    urban_masterplan_data['area_m2'] = urban_masterplan_gdf.geometry.area

    map_1 = KeplerGl(height=600, width=800, mapbox_api_access_token=MAPBOX_ACCESS_TOKEN)
    map_1.add_data(data=geojson_data, name="Neighborhoods")
    map_1.add_data(data=buildings_data, name="Buildings")
    map_1.add_data(data=roads_data, name="Roads")
    map_1.add_data(data=trees_data, name="Trees")
    map_1.add_data(data=transit_data, name="Transit")
    map_1.add_data(data=urban_masterplan_data, name="Urban_Masterplan")

    if not os.path.exists(CONFIG_PATH):
        print("No saved configuration found. Generating a temporary HTML for customization...")
        map_1.save_to_html(file_name=TEMP_HTML_PATH)
        print(f"Opening {TEMP_HTML_PATH} in your browser. Please customize the map, then export the configuration as kepler.gl.json.")
        webbrowser.open(f"file://{os.path.abspath(TEMP_HTML_PATH)}")
        print("After customizing, click the 'Share' button in the Kepler.gl interface, then 'Export Configuration' to save as 'kepler.gl.json' in:")
        print("/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data")
        input("Press Enter once you have saved the configuration file (kepler.gl.json), or Ctrl+C to cancel.")
        if not os.path.exists(CONFIG_PATH):
            raise FileNotFoundError(f"Configuration file {CONFIG_PATH} not found.")

    print(f"Loading configuration from {CONFIG_PATH}...")
    with open(CONFIG_PATH, 'r') as f:
        custom_config = json.load(f)

    for layer in custom_config['config']['visState']['layers']:
        if layer['config']['dataId'] in ["Neighborhoods", "Buildings", "Roads", "Trees", "Transit", "Urban_Masterplan"]:
            layer['config']['isVisible'] = True
        if layer['config']['dataId'] == "Neighborhoods":
            layer['config']['columns'] = {'geojson': 'geometry'}
        elif layer['config']['dataId'] == "Buildings":
            layer['config']['columns'] = {'geojson': 'geometry'}
            layer['visualChannels']['colorField'] = {'name': 'building', 'type': 'string'}
            layer['visualChannels']['heightField'] = {'name': '建物高度', 'type': 'real'}
        elif layer['config']['dataId'] == "Roads":
            layer['config']['columns'] = {'geojson': 'geometry'}
        elif layer['config']['dataId'] == "Trees":
            layer['config']['columns'] = {'geojson': 'geometry'}
        elif layer['config']['dataId'] == "Transit":
            layer['config']['columns'] = {'geojson': 'geometry'}
        elif layer['config']['dataId'] == "Urban_Masterplan":
            layer['config']['columns'] = {'geojson': 'geometry'}
            layer['visualChannels']['colorField'] = {'name': 'Category', 'type': 'string'}
            layer['visualChannels']['opacityField'] = {'name': 'Area', 'type': 'real'}

    map_1 = KeplerGl(height=600, width=800, mapbox_api_access_token=MAPBOX_ACCESS_TOKEN, config=custom_config)
    map_1.add_data(data=geojson_data, name="Neighborhoods")
    map_1.add_data(data=buildings_data, name="Buildings")
    map_1.add_data(data=roads_data, name="Roads")
    map_1.add_data(data=trees_data, name="Trees")
    map_1.add_data(data=transit_data, name="Transit")
    map_1.add_data(data=urban_masterplan_data, name="Urban_Masterplan")

    output_path = os.path.join("/home/johnny/Iaacthesis/projects/Geojson/GNN_Read_data", "walkability_map_city_level.html")
    map_1.save_to_html(file_name=output_path)
    print(f"Interactive Kepler.gl map saved successfully at {output_path}")

    if os.path.exists(TEMP_HTML_PATH):
        os.remove(TEMP_HTML_PATH)
        print(f"Temporary file {TEMP_HTML_PATH} removed.")

def main():
    """Main function to run the walkability analysis pipeline."""
    print("Starting walkability graph network analysis for the entire city...")
    try:
        # Load and prepare data
        data = load_and_prepare_data()

        # Calculate neighborhood features
        data['neighborhoods'], urban_plan_to_neighborhoods = calculate_neighborhood_features(
            data['neighborhoods'], data['trees'], data['transit'], data['urban_masterplan']
        )

        # Build graph
        subgraphs, road_network = build_graph(data, urban_plan_to_neighborhoods)

        # Calculate walkability
        subgraphs = calculate_walkability(subgraphs, data['neighborhoods'])

        # Reconstruct full graph
        G = reconstruct_full_graph(subgraphs, road_network)

        # Create interactive map
        create_interactive_map(G, data)

        print("Analysis completed successfully.")
    except Exception as e:
        print(f"Error during analysis: {e}")
        import traceback
        print(traceback.format_exc())

if __name__ == "__main__":
    main()

Test2